<a href="https://colab.research.google.com/github/JiaxinLi-lipluszn/Temp_notebooks/blob/main/Dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import 

In [31]:
import os
import sys
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import datasets
from torchvision import transforms as pth_transforms
from torchvision import models as torchvision_models
import pandas as pd
from torch.utils.data import Dataset

# Try the ImageFolder dataloader

## Unzip the FashiMNIST into content folder

In [ ]:
!unzip /content/drive/MyDrive/data.zip -d /content/ 
!mv /content/data/test /content/data/val

In [5]:
class ReturnIndexDataset(datasets.ImageFolder):
    def __getitem__(self, idx):
        img, lab = super(ReturnIndexDataset, self).__getitem__(idx)
        return img, idx

In [6]:
transform = pth_transforms.Compose([
        pth_transforms.Resize(256, interpolation=3),
        pth_transforms.CenterCrop(224),
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
dataset_train = ReturnIndexDataset("/content/data/train/", transform=transform)
dataset_val = ReturnIndexDataset("/content/data/val/", transform=transform)
   

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [15]:
x, y = dataset_train[1000]

In [16]:
y

1000

In [11]:
dataset_val.samples[9999]

('/content/data/val/9/9995.png', 9)

# My Implementation

In [17]:
class DINO_Crop(object):
  def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
    self.global_crops_scale = global_crops_scale
    self.local_crops_scale = local_crops_scale
    self.local_crops_number = local_crops_number
    
  def __call__(self, x):
    inputs = []
    data = x  
    #print(data.shape)
    length = x.shape[0]
    
    global_size = int(length * self.global_crops_scale)
    #local_size = int(length * self.local_crops_scale)
    global_index_1 = np.random.choice(length, size = global_size, replace = False)
    #global_index_2 = np.random.choice(length, size = global_size, replace = False)
    
    global_corr_1 = torch.from_numpy(global_index_1)
    #global_corr_2 = torch.from_numpy(global_index_2)
    
    global_data_1 = data[global_index_1,]
    #global_data_2 = data[global_index_2,]
    #print(data.shape)
    #print(corr.shape)
    #labels = torch.from_numpy(label.item()*np.ones(shape = (784,1))).float()
    global_input_1 = torch.cat([global_data_1, global_corr_1]).float()
    #global_input_2 = torch.cat([global_data_2, global_corr_2]).float()
    #nputs.append(global_input_1)
    #inputs.append(global_input_2)
    '''
    for i in range(self.local_crops_number):
      index = np.random.choice(length, size = local_size, replace = False)
      corr = torch.from_numpy(index)
      data_local = data[index,]
      input_local = torch.cat([data_local, corr]).float()
      inputs.append(input_local)
    '''
    return global_input_1

In [18]:
perturb_list = ["A2M", "CCND1", "DAG1", "FBL", "IDH2", "IFNGR1","JAK1", "MIA", "PIK3IP1", "SLC19A1"]
  

In [23]:
expr = pd.DataFrame()
for label in perturb_list:
      path = "/content/drive/MyDrive/scRNA/subset_hvg/train" + "/" + label + ".csv"
      df = pd.read_csv(path, index_col = 0)
      expr = pd.merge(expr, df, how = "outer", left_index = True, right_index = True)
      

In [28]:
len(list(expr.columns))

4014

In [32]:
class scRNACSV(Dataset):
  def __init__(self, expression_matrix_path, perturb_list, transform = DINO_Crop, target_transform=None):
    self.expr = pd.DataFrame()
    self.samples = []
    self.labels = []
    for label in perturb_list:
      path = expression_matrix_path + "/" + label + ".csv"
      df = pd.read_csv(path, index_col = 0)
      self.expr = pd.merge(self.expr, df, how = "outer", left_index = True, right_index = True)
      self.samples = self.samples + list(self.expr.columns)
      lb = [label for i in range(len(list(self.expr.columns)))]
      self.labels = self.labels + lb
    #self.expr = pd.read_csv(expression_matrix_path, index_col= 0)
    self.transform = transform
    self.target_transform = target_transform
    self.samples = [(self.samples[i],self.labels[i]) for i in range(len(self.labels))]
  def __len__(self):
    return self.expr.shape[1]

  def __getitem__(self,idx):
    #print(self.expr.shape)
    one_cell = torch.from_numpy(np.array(self.expr.iloc[:,idx]))
    #print(f"Here is the shape of one_cell {one_cell.shape}")
    ret = self.transform(one_cell)
    #print(len(ret))
    #print(len(label))
    #lab = 1
    return ret, idx

In [33]:
train = scRNACSV("/content/drive/MyDrive/scRNA/subset_hvg/train", perturb_list, transform=transform)
 

In [34]:
train.samples

[('CELL_42982', 'A2M'),
 ('CELL_9744', 'A2M'),
 ('CELL_79203', 'A2M'),
 ('CELL_93928', 'A2M'),
 ('CELL_61444', 'A2M'),
 ('CELL_49466', 'A2M'),
 ('CELL_112865', 'A2M'),
 ('CELL_142647', 'A2M'),
 ('CELL_67074', 'A2M'),
 ('CELL_106938', 'A2M'),
 ('CELL_91884', 'A2M'),
 ('CELL_64497', 'A2M'),
 ('CELL_43505', 'A2M'),
 ('CELL_123202', 'A2M'),
 ('CELL_35750', 'A2M'),
 ('CELL_149245', 'A2M'),
 ('CELL_38432', 'A2M'),
 ('CELL_182955', 'A2M'),
 ('CELL_49138', 'A2M'),
 ('CELL_176320', 'A2M'),
 ('CELL_214174', 'A2M'),
 ('CELL_157517', 'A2M'),
 ('CELL_43774', 'A2M'),
 ('CELL_71660', 'A2M'),
 ('CELL_33939', 'A2M'),
 ('CELL_107191', 'A2M'),
 ('CELL_74837', 'A2M'),
 ('CELL_82174', 'A2M'),
 ('CELL_140322', 'A2M'),
 ('CELL_99741', 'A2M'),
 ('CELL_34110', 'A2M'),
 ('CELL_172306', 'A2M'),
 ('CELL_178336', 'A2M'),
 ('CELL_109160', 'A2M'),
 ('CELL_59296', 'A2M'),
 ('CELL_182363', 'A2M'),
 ('CELL_218281', 'A2M'),
 ('CELL_112000', 'A2M'),
 ('CELL_212493', 'A2M'),
 ('CELL_22866', 'A2M'),
 ('CELL_110999', 'A2M')